# 🏛️ Governance Research Agent

## Building a Citation-Enforced, Forensically-Sound Annual Report Analyser

**Source Documents:** 5 BHP Annual Reports (FY2016-2020)

**Features:**
- PDF extraction with page-level citations
- Citation enforcement on every claim
- Verification agent catches hallucinations
- SHA256 hashes for forensic provenance

---

## Section 1: Environment Setup

### 1.1 Install Dependencies

Uncomment and run if you haven't installed these packages yet:
- `openai-agents` - OpenAI's agent orchestration SDK
- `pydantic` - Data validation and structured outputs
- `pymupdf` - PDF text extraction
- `python-dotenv` - Environment variable management

In [ ]:
# 1.1 - Install dependencies (uncomment if needed)
# !pip install openai-agents pydantic python-dotenv pymupdf

### 1.2 Import Libraries

Import all required libraries:
- Standard library: `os`, `json`, `hashlib`, `re`, `datetime`, `pathlib`
- PDF extraction: `fitz` (PyMuPDF)
- OpenAI Agents SDK: `Agent`, `Runner`, `function_tool`, `trace`
- Pydantic: `BaseModel`, `Field`

In [ ]:
# 1.2 - Import libraries
import os
import json
import hashlib
import re
from datetime import datetime
from pathlib import Path
from typing import Optional

# PDF extraction
import fitz  # PyMuPDF

# OpenAI Agents SDK
from agents import Agent, Runner, function_tool, trace, gen_trace_id

# Pydantic for structured outputs
from pydantic import BaseModel, Field

# Environment variables
from dotenv import load_dotenv

# Jupyter display
from IPython.display import display, Markdown, HTML

print("✅ All imports successful")

### 1.3 Load Environment Variables

Load API keys from `.env` file. Required:
- `OPENAI_API_KEY` - Your OpenAI API key

Make sure you have a `.env` file in your project root with:
```
OPENAI_API_KEY=sk-...
```

In [ ]:
# 1.3 - Load environment variables
load_dotenv(override=True)

if os.environ.get("OPENAI_API_KEY"):
    print("✅ OPENAI_API_KEY is set")
else:
    print("❌ OPENAI_API_KEY is missing! Add it to your .env file")

---

## Section 2: Configuration

### 2.1 Define Paths and PDF Files

Set up paths to the assets folder and define which PDF files to process.

**Your PDFs:**
- BHP_16AR.pdf (2016)
- BHP_17AR.pdf (2017)
- BHP_18AR.pdf (2018)
- BHP_19AR.pdf (2019)
- BHP_20AR.pdf (2020)

In [ ]:
# 2.1 - Define paths and PDF files

# Path to assets folder (relative to notebooks folder)
ASSETS_DIR = Path("../assets")

# List of PDF files with metadata
PDF_FILES = [
    {"filename": "BHP_16AR.pdf", "company": "BHP", "year": 2016},
    {"filename": "BHP_17AR.pdf", "company": "BHP", "year": 2017},
    {"filename": "BHP_18AR.pdf", "company": "BHP", "year": 2018},
    {"filename": "BHP_19AR.pdf", "company": "BHP", "year": 2019},
    {"filename": "BHP_20AR.pdf", "company": "BHP", "year": 2020},
]

# Verify PDFs exist
print("📁 Checking PDF files:")
for pdf in PDF_FILES:
    path = ASSETS_DIR / pdf["filename"]
    if path.exists():
        size_mb = path.stat().st_size / (1024 * 1024)
        print(f"   ✅ {pdf['filename']} ({size_mb:.1f} MB)")
    else:
        print(f"   ❌ {pdf['filename']} NOT FOUND at {path}")

### 2.2 Model and Processing Settings

Configure:
- **Models**: Which GPT models to use (gpt-4o-mini for most tasks, gpt-4o for verification)
- **Chunking**: How to split PDF text (1000 chars with 200 char overlap)
- **Agent settings**: Max turns before timeout

In [ ]:
# 2.2 - Model and processing settings

CONFIG = {
    # Models
    "default_model": "gpt-4o-mini",  # Fast, cheap - for most tasks
    "strong_model": "gpt-4o",         # Stronger - for verification
    
    # Chunking settings
    "chunk_size": 1000,      # Characters per chunk
    "chunk_overlap": 200,    # Overlap between chunks
    
    # Agent settings
    "max_turns": 25,         # Max tool calls before stopping
    "max_iterations": 3,     # Max write-verify loops
}

print("✅ Configuration loaded")
print(f"   Default model: {CONFIG['default_model']}")
print(f"   Chunk size: {CONFIG['chunk_size']} chars")
print(f"   Max turns: {CONFIG['max_turns']}")

---

## Section 3: Data Models (Pydantic)

### 3.1 Source Document Models

Define Pydantic models for tracking PDF sources and text chunks:

- **PDFSource**: Metadata about each PDF (filename, hash, page count)
- **DocumentChunk**: A piece of extracted text with its location (page, section)

In [ ]:
# 3.1 - Source document models

class PDFSource(BaseModel):
    """Metadata about a source PDF document."""
    filename: str
    company: str
    year: int
    document_type: str = "Annual Report"
    sha256_hash: str           # For provenance verification
    total_pages: int
    file_path: str


class DocumentChunk(BaseModel):
    """A chunk of text extracted from a PDF with full provenance."""
    chunk_id: str              # Unique ID: bhp_2019_p087_c003
    content: str               # The actual text
    company: str
    year: int
    page: int                  # Page number (1-indexed)
    section: str = "General"   # Detected section name
    source_file: str


print("✅ Source document models defined")

### 3.2 Agent Output Models

Define structured outputs for each agent:

- **ClarifyingQuestions**: Output from Clarifier Agent
- **AnalystFinding**: A single finding with citation
- **AnalystReport**: Complete output from Analyst Agent
- **ReportData**: Output from Writer Agent

In [ ]:
# 3.2 - Agent output models

class ClarifyingQuestions(BaseModel):
    """Output from the Clarifier Agent."""
    questions: list[str]           # Clarifying questions
    refined_query: str             # Improved query
    identified_companies: list[str]
    identified_years: list[int]
    focus_areas: list[str]         # e.g., ["remuneration", "board"]


class AnalystFinding(BaseModel):
    """A single finding with citation."""
    finding: str                   # The factual finding
    evidence: str                  # Quote from source
    chunk_id: str                  # Source chunk ID
    page: int                      # Page number
    confidence: str = "high"       # high, medium, low
    category: str                  # remuneration, board, etc.


class AnalystReport(BaseModel):
    """Complete output from an Analyst Agent."""
    analyst_name: str
    findings: list[AnalystFinding]
    summary: str
    red_flags: list[str] = []


class ReportData(BaseModel):
    """Output from the Writer Agent."""
    short_summary: str             # 2-3 sentence summary
    markdown_report: str           # Full report
    claims_made: list[str]         # List of factual claims
    follow_up_questions: list[str]


print("✅ Agent output models defined")

### 3.3 Verification Models

Models for the verification process:

- **ClaimVerification**: Result for a single claim (VERIFIED, UNSUPPORTED, HALLUCINATION)
- **VerificationResult**: Complete verification report

In [ ]:
# 3.3 - Verification models

class ClaimVerification(BaseModel):
    """Verification result for a single claim."""
    claim: str
    status: str                    # VERIFIED, UNSUPPORTED, HALLUCINATION
    source_chunk_id: Optional[str] = None
    source_text: Optional[str] = None
    page: Optional[int] = None
    reason: Optional[str] = None


class VerificationResult(BaseModel):
    """Complete verification result."""
    verified: bool                 # True if all claims pass
    total_claims: int
    verified_count: int
    claims: list[ClaimVerification]
    hallucinations: list[str] = []
    unsupported: list[str] = []


print("✅ Verification models defined")

---

## Section 4: PDF Extraction

### 4.1 Helper Functions

Define helper functions for:
- **compute_sha256**: Calculate file hash for provenance
- **detect_section**: Guess which section of the annual report a chunk is from

In [ ]:
# 4.1 - Helper functions

def compute_sha256(file_path: Path) -> str:
    """Compute SHA256 hash of a file for provenance tracking."""
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            sha256_hash.update(chunk)
    return sha256_hash.hexdigest()


def detect_section(text: str) -> str:
    """Detect which section of the annual report this text is from."""
    text_lower = text.lower()
    
    if any(term in text_lower for term in ["remuneration", "compensation", "executive pay", "kmp", "key management"]):
        return "Remuneration Report"
    elif any(term in text_lower for term in ["director", "board", "chairman", "governance", "committee"]):
        return "Directors Report"
    elif any(term in text_lower for term in ["related party", "transactions with"]):
        return "Related Party Disclosures"
    elif any(term in text_lower for term in ["financial statements", "balance sheet", "income statement"]):
        return "Financial Statements"
    elif any(term in text_lower for term in ["risk", "material risk"]):
        return "Risk Management"
    elif any(term in text_lower for term in ["sustainability", "environment", "climate", "emissions", "safety"]):
        return "Sustainability"
    else:
        return "General"


print("✅ Helper functions defined")

### 4.2 PDF Extraction Function

Main function to extract text from a PDF and split into chunks:

1. Open PDF with PyMuPDF
2. Extract text from each page
3. Split into chunks of ~1000 characters
4. Create chunk IDs with format: `bhp_2019_p087_c003`
5. Detect section for each chunk
6. Calculate SHA256 hash for provenance

In [ ]:
# 4.2 - PDF extraction function

def extract_pdf_chunks(
    pdf_path: Path, 
    company: str, 
    year: int,
    chunk_size: int = 1000,
    chunk_overlap: int = 200
) -> tuple[PDFSource, list[DocumentChunk]]:
    """
    Extract text from a PDF and split into chunks with page citations.
    
    Args:
        pdf_path: Path to PDF file
        company: Company code (e.g., "BHP")
        year: Financial year
        chunk_size: Target characters per chunk
        chunk_overlap: Overlap between chunks
    
    Returns:
        Tuple of (PDFSource metadata, list of DocumentChunks)
    """
    doc = fitz.open(pdf_path)
    
    # Create source metadata
    source = PDFSource(
        filename=pdf_path.name,
        company=company,
        year=year,
        sha256_hash=compute_sha256(pdf_path),
        total_pages=len(doc),
        file_path=str(pdf_path)
    )
    
    chunks = []
    chunk_counter = 0
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text()
        
        if not text.strip():
            continue
        
        # Split page text into chunks
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk_text = text[start:end]
            
            # Try to break at sentence boundary
            if end < len(text):
                last_period = chunk_text.rfind('.')
                if last_period > chunk_size // 2:
                    chunk_text = chunk_text[:last_period + 1]
                    end = start + last_period + 1
            
            if chunk_text.strip():
                # Create unique chunk ID
                chunk_id = f"{company.lower()}_{year}_p{page_num + 1:03d}_c{chunk_counter:03d}"
                
                chunks.append(DocumentChunk(
                    chunk_id=chunk_id,
                    content=chunk_text.strip(),
                    company=company,
                    year=year,
                    page=page_num + 1,  # 1-indexed
                    section=detect_section(chunk_text),
                    source_file=str(pdf_path)
                ))
                chunk_counter += 1
            
            start = end - chunk_overlap
    
    doc.close()
    return source, chunks


print("✅ PDF extraction function defined")

### 4.3 Extract All PDFs

Process all PDF files and build the chunk database.

**This cell takes ~1-2 minutes** to process all 5 PDFs.

Output:
- `PDF_SOURCES`: Dict of PDFSource objects (metadata)
- `ALL_CHUNKS`: Dict of DocumentChunk objects (searchable text)

In [ ]:
# 4.3 - Extract all PDFs

print("📄 Extracting text from PDFs...")
print("   (This takes ~1-2 minutes for 5 large PDFs)\n")

PDF_SOURCES: dict[str, PDFSource] = {}
ALL_CHUNKS: dict[str, DocumentChunk] = {}

for pdf_info in PDF_FILES:
    pdf_path = ASSETS_DIR / pdf_info["filename"]
    
    if not pdf_path.exists():
        print(f"   ⚠️ Skipping {pdf_info['filename']} - not found")
        continue
    
    print(f"   📖 Processing {pdf_info['filename']}...", end=" ", flush=True)
    
    source, chunks = extract_pdf_chunks(
        pdf_path,
        company=pdf_info["company"],
        year=pdf_info["year"],
        chunk_size=CONFIG["chunk_size"],
        chunk_overlap=CONFIG["chunk_overlap"]
    )
    
    PDF_SOURCES[pdf_info["filename"]] = source
    for chunk in chunks:
        ALL_CHUNKS[chunk.chunk_id] = chunk
    
    print(f"{len(chunks)} chunks from {source.total_pages} pages")

print(f"\n✅ Extraction complete!")
print(f"   Total chunks: {len(ALL_CHUNKS):,}")
print(f"   Total PDFs: {len(PDF_SOURCES)}")

### 4.4 Inspect Extracted Chunks

View sample chunks to verify extraction worked correctly.

We'll look at:
- Section distribution (how many chunks per section)
- Sample remuneration chunks (most relevant for governance)

In [ ]:
# 4.4 - Inspect extracted chunks

# Count chunks by section
section_counts = {}
for chunk in ALL_CHUNKS.values():
    section = chunk.section
    section_counts[section] = section_counts.get(section, 0) + 1

print("📊 Chunks by section:")
for section, count in sorted(section_counts.items(), key=lambda x: -x[1]):
    print(f"   {section}: {count:,}")

# Show sample remuneration chunks
print("\n📋 Sample Remuneration chunks:")
print("-" * 60)

rem_chunks = [c for c in ALL_CHUNKS.values() if c.section == "Remuneration Report"][:3]
for chunk in rem_chunks:
    print(f"\nChunk ID: {chunk.chunk_id}")
    print(f"Source: {chunk.company} {chunk.year} AR, Page {chunk.page}")
    print(f"Content: {chunk.content[:200]}...")
    print("-" * 60)

---

## Section 5: Search Tools

### 5.1 Main Search Tool

The `search_annual_reports` tool searches across all extracted chunks:

- Keyword matching with scoring
- Filter by company, year, section
- Returns top results with citations

**Important:** This is what agents use to find information in the PDFs.

In [ ]:
# 5.1 - Main search tool

def _search_annual_reports(
    query: str, 
    company: str = "BHP",
    year: int = None,
    section: str = None,
    max_results: int = 5
) -> str:
    """Core search logic (not decorated)."""
    query_terms = [t.lower() for t in query.split() if len(t) > 2]
    results = []
    
    for chunk_id, chunk in ALL_CHUNKS.items():
        if chunk.company.upper() != company.upper():
            continue
        if year and chunk.year != year:
            continue
        if section and section.lower() not in chunk.section.lower():
            continue
        
        content_lower = chunk.content.lower()
        score = 0
        for term in query_terms:
            if term in content_lower:
                score += 2
                score += content_lower.count(term) * 0.5
        
        if score > 0:
            results.append((score, chunk))
    
    results.sort(key=lambda x: x[0], reverse=True)
    
    if not results:
        return f"No results found for '{query}' in {company} {year or 'all years'}"
    
    formatted = [f"Found {len(results)} results. Top {min(max_results, len(results))}:\n"]
    for score, chunk in results[:max_results]:
        content = chunk.content[:600] + "..." if len(chunk.content) > 600 else chunk.content
        formatted.append(f"""
---
**[{chunk.company} {chunk.year} AR, Page {chunk.page}]**
Section: {chunk.section}

{content}

[Chunk ID: {chunk.chunk_id}]
""")
    
    return "\n".join(formatted)


# Wrap for agents
@function_tool
def search_annual_reports(
    query: str, 
    company: str = "BHP",
    year: int = None,
    section: str = None,
    max_results: int = 5
) -> str:
    """Search annual reports. Returns passages with citations."""
    return _search_annual_reports(query, company, year, section, max_results)


print("✅ search_annual_reports tool defined")

### 5.2 Additional Tools

Helper tools for the agents:

- **get_chunk_by_id**: Retrieve a specific chunk (for verification)
- **list_available_years**: Show what years are available
- **get_section_summary**: Get chunks from a specific section

In [ ]:
# 5.2 - Additional tools

def _get_chunk_by_id(chunk_id: str) -> str:
    if chunk_id not in ALL_CHUNKS:
        return f"ERROR: Chunk ID '{chunk_id}' not found."
    chunk = ALL_CHUNKS[chunk_id]
    return f"""
**Chunk ID:** {chunk.chunk_id}
**Source:** {chunk.company} {chunk.year} AR, Page {chunk.page}
**Section:** {chunk.section}

**Content:**
{chunk.content}
"""

@function_tool
def get_chunk_by_id(chunk_id: str) -> str:
    """Retrieve a chunk by ID."""
    return _get_chunk_by_id(chunk_id)


def _list_available_years(company: str = "BHP") -> str:
    year_counts = {}
    for chunk in ALL_CHUNKS.values():
        if chunk.company.upper() == company.upper():
            year_counts[chunk.year] = year_counts.get(chunk.year, 0) + 1
    if not year_counts:
        return f"No documents for {company}"
    return f"Available: {sorted(year_counts.keys())}"

@function_tool
def list_available_years(company: str = "BHP") -> str:
    """List available years."""
    return _list_available_years(company)


def _get_section_summary(section: str, company: str = "BHP", year: int = None, max_chunks: int = 3) -> str:
    matching = [c for c in ALL_CHUNKS.values() 
                if c.company.upper() == company.upper() 
                and section.lower() in c.section.lower()
                and (year is None or c.year == year)]
    if not matching:
        return f"No '{section}' chunks found"
    result = f"Found {len(matching)}. Sample:\n"
    for c in matching[:max_chunks]:
        result += f"\n[{c.year} Page {c.page}] {c.content[:300]}...\n[Chunk ID: {c.chunk_id}]\n"
    return result

@function_tool
def get_section_summary(section: str, company: str = "BHP", year: int = None, max_chunks: int = 3) -> str:
    """Get chunks from a section."""
    return _get_section_summary(section, company, year, max_chunks)

print("✅ Additional tools defined")

### 5.3 Test Search Tool

Test the search tool to make sure it's finding relevant content.

We'll search for CEO remuneration information.

In [ ]:
# 5.3 - Test search tool

print("🔍 Test search: 'CEO total remuneration pay'\n")
print("=" * 60)

result = _search_annual_reports(
    query="CEO total remuneration pay",
    company="BHP",
    max_results=3
)
print(result)

---

## Section 6: Agent Definitions

### 6.1 Clarifier Agent

The Clarifier Agent:
- Asks clarifying questions about the research query
- Identifies companies and years mentioned
- Refines the query with sensible defaults

In [ ]:
# 6.1 - Clarifier Agent

CLARIFIER_INSTRUCTIONS = """
You are a governance research assistant specialising in ASX-listed companies.

Given a research query, your job is to:
1. Generate 3 clarifying questions that would help focus the research
2. Identify the companies mentioned or implied
3. Identify the time period (years) to analyse
4. Determine the governance focus areas (remuneration, board, related parties, etc.)
5. Provide a refined query with sensible defaults assumed

AVAILABLE DATA: BHP Annual Reports for years 2016, 2017, 2018, 2019, 2020.

If no years specified, default to 2019 and 2020 (most recent).
If no company specified, default to BHP.

Be concise and practical.
"""

clarifier_agent = Agent(
    name="ClarifierAgent",
    instructions=CLARIFIER_INSTRUCTIONS,
    model=CONFIG["default_model"],
    output_type=ClarifyingQuestions,
)

print("✅ Clarifier Agent defined")

### 6.2 Remuneration Analyst Agent

The Analyst Agent:
- Searches the annual reports using the tools
- Extracts specific findings about executive remuneration
- **MUST cite every finding** with a chunk_id

**Critical:** This agent can only use information from the search tools.

In [ ]:
# 6.2 - Remuneration Analyst Agent

REMUNERATION_ANALYST_INSTRUCTIONS = """
You analyse BHP annual reports for executive remuneration.

RULES:
1. Use search_annual_reports to find information
2. Every finding MUST have a chunk_id from the search results
3. Stop after finding 3-5 good findings

WORKFLOW:
1. Search for "CEO remuneration salary total" 
2. Extract findings with chunk_id citations
3. Return your AnalystReport

DO NOT keep searching indefinitely. After 2-3 searches, compile your findings.
"""

remuneration_analyst = Agent(
    name="RemunerationAnalyst",
    instructions=REMUNERATION_ANALYST_INSTRUCTIONS,
    tools=[search_annual_reports, get_chunk_by_id, list_available_years, get_section_summary],
    model=CONFIG["default_model"],
    output_type=AnalystReport,
)

print("✅ Remuneration Analyst Agent defined")

### 6.3 Writer Agent

The Writer Agent:
- Takes findings from the analyst
- Synthesises them into a professional report
- **MUST cite every claim** with [Chunk ID: xxx]

In [ ]:
# 6.3 - Writer Agent

WRITER_INSTRUCTIONS = """
You are a senior governance report writer.

Your job is to synthesise analyst findings into a professional report.

CRITICAL CONSTRAINTS:
1. Use ONLY information from the provided findings
2. Do NOT add any external knowledge
3. EVERY factual claim MUST include [Chunk ID: xxx] citation
4. If something isn't in the findings, say "Not available in source documents"

FORBIDDEN:
- "Generally, companies..." (external knowledge)
- "It is common practice..." (external knowledge)
- Any claim without [Chunk ID: xxx] citation

REQUIRED FORMAT:
"The CEO total remuneration was $X [Chunk ID: bhp_2019_p087_c003]."

REPORT STRUCTURE:
1. Executive Summary (2-3 sentences with citations)
2. Key Findings (bullet points with citations)
3. Detailed Analysis (paragraphs with citations)
4. Red Flags (if any)
5. Recommendations for Further Review

Write in professional, clear language.
"""

writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_INSTRUCTIONS,
    model=CONFIG["default_model"],
    output_type=ReportData,
)

print("✅ Writer Agent defined")

### 6.4 Verifier Agent

The Verifier Agent:
- Checks every claim in the report
- Verifies the cited chunk_id exists
- Confirms the claim matches the source

Uses the stronger model (gpt-4o) for accuracy.

In [ ]:
# 6.4 - Verifier Agent

VERIFIER_INSTRUCTIONS = """
You are a verification agent ensuring forensic integrity of governance reports.

Your job is to verify that EVERY factual claim:
1. Has a citation [Chunk ID: xxx]
2. The chunk_id exists (use get_chunk_by_id to check)
3. The claim accurately reflects the source

For each claim, assign status:
- VERIFIED: Has valid citation that supports the claim
- UNSUPPORTED: Citation exists but doesn't support claim
- HALLUCINATION: No citation or invalid chunk_id

Be STRICT. This is for legal/regulatory use.

Output:
- verified: true only if ALL claims are VERIFIED
- List each claim with its status
- List any hallucinations
- List any unsupported claims
"""

verifier_agent = Agent(
    name="VerifierAgent",
    instructions=VERIFIER_INSTRUCTIONS,
    tools=[get_chunk_by_id],
    model=CONFIG["strong_model"],  # Use stronger model for verification
    output_type=VerificationResult,
)

print("✅ Verifier Agent defined")

---

## Section 7: Test the Analyst Agent

### 7.1 Run Analyst Test

Test the remuneration analyst on a simple query.

**Note:** `max_turns=25` allows the agent more iterations to search.

In [ ]:
# 7.1 - Run analyst test

print("=" * 60)
print("TEST: Remuneration Analyst Agent")
print("=" * 60)

analyst_query = """Search for CEO remuneration information in the BHP 2019 and 2020 annual reports.
Find the total pay figures and any year-on-year changes."""

print(f"Query: {analyst_query}\n")

with trace("Analyst Test"):
    result = await Runner.run(
        remuneration_analyst, 
        analyst_query,
        max_turns=CONFIG["max_turns"]  # Allow more turns
    )
    analysis = result.final_output_as(AnalystReport)

print(f"\n📊 Analyst: {analysis.analyst_name}")
print(f"\n📝 Summary: {analysis.summary}")

### 7.2 Display Analyst Findings

Show the detailed findings with citations.

In [ ]:
# 7.2 - Display analyst findings

print(f"\n🔍 Findings ({len(analysis.findings)}):")
print("=" * 60)

for i, finding in enumerate(analysis.findings, 1):
    print(f"\n📌 Finding {i}:")
    print(f"   Statement: {finding.finding}")
    print(f"   Evidence: {finding.evidence[:200]}..." if len(finding.evidence) > 200 else f"   Evidence: {finding.evidence}")
    print(f"   Citation: [Chunk ID: {finding.chunk_id}], Page {finding.page}")
    print(f"   Confidence: {finding.confidence}")

if analysis.red_flags:
    print(f"\n🚩 Red Flags:")
    for flag in analysis.red_flags:
        print(f"   - {flag}")

In [ ]:
# Verify Finding 1 manually
chunk_id = "bhp_2019_p152_c1033"
print(_get_chunk_by_id(chunk_id))

---

## Section 8: Full Research Pipeline

### 8.1 Research Manager Class

The Research Manager orchestrates the full pipeline:

1. **Clarify** → Refine the query
2. **Analyse** → Extract findings with citations
3. **Write** → Create report
4. **Verify** → Check all citations

In [ ]:
# 8.1 - Research Manager class

class GovernanceResearchManager:
    """
    Orchestrates the full governance research pipeline:
    Clarify → Analyse → Write → Verify
    """
    
    def __init__(self):
        self.max_turns = CONFIG["max_turns"]
        self.chunks_retrieved = []
    
    async def run(self, query: str):
        """Run the full pipeline. Yields status updates."""
        trace_id = gen_trace_id()
        
        with trace("Governance Research", trace_id=trace_id):
            yield f"🔗 Trace: https://platform.openai.com/traces/{trace_id}"
            
            # Step 1: Clarify
            yield "\n💭 Step 1: Clarifying query..."
            clarification = await self._clarify(query)
            yield f"   ✅ Refined: {clarification.refined_query[:80]}..."
            yield f"   Years: {clarification.identified_years}"
            yield f"   Focus: {clarification.focus_areas}"
            
            # Step 2: Analyse
            yield "\n🔍 Step 2: Analyst searching reports..."
            findings = await self._analyse(clarification)
            yield f"   ✅ Found {len(findings)} findings"
            
            # Step 3: Write
            yield "\n📝 Step 3: Writing report..."
            report = await self._write(clarification.refined_query, findings)
            yield "   ✅ Report drafted"
            
            # Step 4: Verify
            yield "\n🔬 Step 4: Verifying citations..."
            verification = await self._verify(report)
            
            if verification.verified:
                yield f"   ✅ PASSED: {verification.verified_count}/{verification.total_claims} claims verified"
            else:
                yield f"   ⚠️ ISSUES: {len(verification.hallucinations)} hallucinations, {len(verification.unsupported)} unsupported"
            
            yield "\n✅ Research complete!"
            
            # Return results
            yield {
                "query": query,
                "refined_query": clarification.refined_query,
                "trace_id": trace_id,
                "report": report,
                "verification": verification,
                "findings": findings,
                "sources": list(PDF_SOURCES.values()),
                "chunks_used": self.chunks_retrieved,
            }
    
    async def _clarify(self, query: str) -> ClarifyingQuestions:
        """Step 1: Clarify the query."""
        result = await Runner.run(
            clarifier_agent, 
            f"Query: {query}",
            max_turns=5
        )
        return result.final_output_as(ClarifyingQuestions)
    
    async def _analyse(self, clarification: ClarifyingQuestions) -> list[AnalystFinding]:
        """Step 2: Run the analyst."""
        years = clarification.identified_years or [2019, 2020]
        focus = clarification.focus_areas or ["remuneration"]
        
        analyst_prompt = f"""Search the BHP annual reports for {', '.join(map(str, years))}.
        Focus on: {', '.join(focus)}
        
        Find specific information and cite every finding with chunk_id."""
        
        result = await Runner.run(
            remuneration_analyst, 
            analyst_prompt,
            max_turns=self.max_turns
        )
        analysis = result.final_output_as(AnalystReport)
        
        # Track chunks used
        for f in analysis.findings:
            if f.chunk_id not in self.chunks_retrieved:
                self.chunks_retrieved.append(f.chunk_id)
        
        return analysis.findings
    
    async def _write(self, query: str, findings: list[AnalystFinding]) -> ReportData:
        """Step 3: Write the report."""
        findings_text = "\n\n".join([
            f"Finding: {f.finding}\n"
            f"Evidence: {f.evidence}\n"
            f"Citation: [Chunk ID: {f.chunk_id}], Page {f.page}\n"
            f"Confidence: {f.confidence}"
            for f in findings
        ])
        
        prompt = f"""Query: {query}

Analyst Findings:
{findings_text}

Write a report using ONLY these findings. Cite every claim with [Chunk ID: xxx]."""
        
        result = await Runner.run(writer_agent, prompt, max_turns=5)
        return result.final_output_as(ReportData)
    
    async def _verify(self, report: ReportData) -> VerificationResult:
        """Step 4: Verify the report."""
        prompt = f"""Verify this report. Check each claim has a valid [Chunk ID: xxx] citation.
Use get_chunk_by_id to verify chunk_ids exist.

Report:
{report.markdown_report}

Claims made: {report.claims_made}"""
        
        result = await Runner.run(
            verifier_agent, 
            prompt,
            max_turns=self.max_turns
        )
        return result.final_output_as(VerificationResult)


print("✅ GovernanceResearchManager defined")

### 8.2 Run Full Pipeline

Execute the complete research pipeline.

**This takes 1-3 minutes** depending on API response times.

In [ ]:
# 8.2 - Run full pipeline

manager = GovernanceResearchManager()

query = "Analyse BHP executive remuneration trends from 2018 to 2020"

print("=" * 60)
print("GOVERNANCE RESEARCH PIPELINE")
print("=" * 60)
print(f"Query: {query}")
print("=" * 60)

output = None
async for update in manager.run(query):
    if isinstance(update, str):
        print(update)
    else:
        output = update

print("\n" + "=" * 60)

### 8.3 Display Report

Show the final report with verification status.

In [ ]:
# 8.3 - Display report

if output:
    print("\n📊 EXECUTIVE SUMMARY")
    print("-" * 40)
    print(output["report"].short_summary)
    
    print("\n📋 VERIFICATION STATUS")
    print("-" * 40)
    v = output["verification"]
    status = "✅ PASSED" if v.verified else "⚠️ ISSUES FOUND"
    print(f"Status: {status}")
    print(f"Claims verified: {v.verified_count}/{v.total_claims}")
    
    if v.hallucinations:
        print(f"\n⚠️ Hallucinations: {v.hallucinations}")
    if v.unsupported:
        print(f"\n⚠️ Unsupported: {v.unsupported}")
    
    print("\n📄 FULL REPORT")
    print("-" * 40)
    display(Markdown(output["report"].markdown_report))
else:
    print("❌ No output - check for errors above")

### 8.4 Provenance Trail

Display the complete audit trail:
- PDF sources with SHA256 hashes
- Chunks retrieved
- Trace link for debugging

In [ ]:
# 8.4 - Provenance trail

if output:
    print("\n🔒 PROVENANCE TRAIL")
    print("=" * 60)
    
    print("\n📁 PDF Sources:")
    for source in output["sources"]:
        print(f"   • {source.filename}")
        print(f"     Year: {source.year}, Pages: {source.total_pages}")
        print(f"     SHA256: {source.sha256_hash[:40]}...")
    
    print(f"\n📎 Chunks Used ({len(output['chunks_used'])}):")
    for chunk_id in output['chunks_used'][:10]:
        if chunk_id in ALL_CHUNKS:
            c = ALL_CHUNKS[chunk_id]
            print(f"   • {chunk_id} (Page {c.page}, {c.section})")
    if len(output['chunks_used']) > 10:
        print(f"   ... and {len(output['chunks_used']) - 10} more")
    
    print(f"\n🔗 Trace: https://platform.openai.com/traces/{output['trace_id']}")

---

## Section 9: Summary

### What We Built

| Component | Purpose |
|-----------|--------|
| PDF Extraction | PyMuPDF extracts text with page numbers |
| Chunking | ~1000 char chunks with 200 char overlap |
| Search Tool | Keyword search with scoring |
| Clarifier Agent | Refines query, identifies scope |
| Analyst Agent | Extracts findings with citations |
| Writer Agent | Creates report with mandatory citations |
| Verifier Agent | Checks all claims are supported |

### Next Steps

1. Add vector search (embeddings) for better retrieval
2. Add Cohere reranking
3. Build Gradio UI with PDF viewer
4. Add supplementary web search (ASIC, ASX)

In [ ]:
# 9.1 - Final summary

print("=" * 60)
print("🎉 NOTEBOOK COMPLETE")
print("=" * 60)
print(f"""
You've built a governance research agent with:

✅ PDF extraction from {len(PDF_SOURCES)} BHP annual reports
✅ {len(ALL_CHUNKS):,} searchable chunks with page citations
✅ Citation enforcement [Chunk ID: xxx] on every claim
✅ Verification agent to catch hallucinations
✅ SHA256 hashes for forensic provenance

Source documents: BHP Annual Reports 2016-2020
""")